## Determing the number of hidden layers and neurons by hyperparameter tuning

### This can be challenging and often requires experimentation. however there are some guidelines and 
### methods that can help you in making an informed decision.
### Start Simple : Begin with a simple architecture and gradually increase complexity if needed 
### Grid Search/Random Search : Use GridSearch or RandomSearch to try out different architectures
### Cross Validation : Use CV to evaluate the performance of difference of diffrent architectures 
### Heuristic and Rules of Thumb : 
### 1. Number of neurons in hidden layers should be between the size of the input layer and the size 
### of the output layer
### 2. A common practice is to start with 1-2 hidden layers

In [6]:
!pip install scikeras

  Using cached scikeras-0.13.0-py3-none-any.whl.metadata (3.1 kB)
  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached rich-14.0.0-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.9-py3-none-any.whl.metadata (322 bytes)
  Using cached optree-0.15.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (48 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached scikeras-0.13.0-py3-none-any.whl (26 kB)
Using cached keras-3.10.0-py3-none-any.whl (1.4 MB)
Using cached namex-0.0.9-py3-none-any.whl (5.8 kB)
Using cached optree-0.15.0-cp311-cp311-macosx_11_0_arm64.whl (338 kB)
Using cached rich-14.0.0-py3-none-any.whl (243 kB)
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
  Attempting uninstall: keras━╸━━━━━━━━━━━━━━━━━ 4/7 [rich]
    Found existing installation: keras 2.15.0━━━━━━━━━━━━━━━━━ 4/7 [rich]
    Unins

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

ModuleNotFoundError: No module named 'tensorflow.keras'

In [13]:
data = pd.read_csv('Churn_Modelling.csv')
data = data.drop(columns=['RowNumber','CustomerId','Surname'],axis=1)
scaler = StandardScaler()
one_hot_encoder_geo = OneHotEncoder()
lable_encoder_gender = LabelEncoder()
data['Gender'] = lable_encoder_gender.fit_transform(data['Gender'])
geo_encoded = one_hot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
x = data.drop('Exited',axis=1)
y = data['Exited']

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## dump all the models scalers as pickle files 



In [14]:
## define a fucntion to create the model and try different parameters (KerasClassifier)

def create_model(neurons=32,layers=1):
    model = Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(x_train.shape[1],)))
    
    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [ ]:
## create a kerasclassifier
model = KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [16]:
## define the grid search parameters 
param_grid = {
    'neurons' : [16,32,48,128],
    'layers' : [1,2],
    'epochs' : [50,100]
}

In [ ]:
## Perform grid search
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3)
grid_result = grid.fit(x_train,y_train)

## print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))